# Valor Presente y Sensibilidad OIS

**Definición:** una curva cupón cero es una colección ordenada de plazos y tasas de interés donde cada una de las tasas es una  tasa adecuada para traer a valor presente un flujo de caja al plazo que corresponde a la tasa.

Consideremos un instrumento financiero cuyo valor depende de una curva cupón cero. Por ejemplo, un swap de tasa de interés. Matemáticamente, esta dependencia se expresa como:

$$
V = V\left(z_1,\ldots,z_n;\alpha\right)
$$

donde $z_1,\ldots,z_N$ son los valores de las tasas de la curva y $\alpha$ representa un vector de parámetros adicionales (como la tasa de cupón en un bono o un tipo de cambio en un swap de monedas).

**Definición:** la cantidad $\Delta_i$ de $V$ respecto a $z_i$ se define como:

$$
\Delta_i=\frac{\partial V}{\partial z_i}
$$

**Definición:** la sensibilidad de $V$ respecto a un cambio $\delta z_i$ (positivo o negativo) en la tasa $z_i$ se define como:

$$
S\left(V,z_i,\delta_i\right)=V\left(z_1,...,z_i+\delta_i,\ldots,z_n\right)-V\left(z_1,\ldots,z_i,\ldots,z_n\right)
$$

Cuando se busca conocer la sensibilidad a un cambio $\delta z_i$  en la tasa $z_i$, donde $\delta z_i$ representa una magnitud sin signo, se utiliza la siguiente definición:

**Definición:** la sensibilidad de $V$ respecto a un cambio de magnitud $\delta z_i$ en la tasa $z_i$ se define como:

$$
\overline{S}\left(V,z_i,\delta_i\right)=\frac{V\left(z_1,\ldots ,z_i+\delta_i,\ldots,z_n\right)-V\left(z_1,\ldots,z_i-\delta_i,...,z_n\right)}{2}
$$

En ocasiones, cuando se busca conocer la sensibilidad a un movimiento de magnitud pequeña, resulta conveniente realizar la siguiente aproximación:

$$
\overline{S}\left(V,z_i,\delta_i\right)\approx\Delta_i \cdot\delta_i
$$

este es el caso, cuando se busca conocer esta sensibilidad para un elevado número de operaciones. Es importante considerar que se puede asegurar que esta aproximación es válida cuando la función $V$ satisface los requerimientos de regularidad necesarios para aplicar el [Teorema de Taylor](https://en.wikipedia.org/wiki/Taylor%27s_theorem).


## Configuración

### Librerías

In [17]:
from finrisk import QC_Financial_3 as Qcf
from functools import partial
from enum import Enum
import pandas as pd
import my_functions as my

### Variables Globales

In [18]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [19]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    Retorna un calendario NY o SCL.
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    if code == BusCal.SCL:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            cal.add_holiday(Qcf.QCDate(1, 1, agno))
            cal.add_holiday(Qcf.QCDate(18, 9, agno))
            cal.add_holiday(Qcf.QCDate(19, 9, agno))
            cal.add_holiday(Qcf.QCDate(25, 12, agno))
        
    return cal

In [20]:
get_cal(BusCal.NY)

In [21]:
frmt = {
    'tasa': '{:.6%}',
    'df': '{:.6%}',
    'valor_tasa': '{:.4%}',
    'spread': '{:.4%}',
    'nominal': '{:,.0f}',
    'interes': '{:,.0f}',
    'amortizacion': '{:,.0f}',
    'flujo': '{:,.4f}',
}

In [22]:
class TypeOis(Enum):
    SOFR = 1
    ICP = 2

In [23]:
type_ois_template = {
    TypeOis.SOFR: {
        'currency': Qcf.QCUSD(),
        'periodicity': Qcf.Tenor('1Y'),
        'stub_period': Qcf.StubPeriod.SHORTFRONT,
        'settlement_lag': 0,
        'calendar': BusCal.NY,
        'bus_adj_rule': Qcf.BusyAdjRules.MODFOLLOW,
        'amort_is_cashflow': True,
        'fixed_rate': Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    },
    TypeOis.ICP: {
        'currency': Qcf.QCCLP(),
        'periodicity': Qcf.Tenor('6M'),
        'stub_period': Qcf.StubPeriod.SHORTFRONT,
        'settlement_lag': 0,
        'calendar': BusCal.SCL,
        'bus_adj_rule': Qcf.BusyAdjRules.MODFOLLOW,
        'amort_is_cashflow': True,
        'fixed_rate': Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    }
}

## Construye Curva Cero Cupón

Se importa la data de una curva cupón cero construida con cotizaciones SOFR.

In [24]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [25]:
def get_curve_from_dataframe(yf: Qcf.QCYearFraction, wf: Qcf.QCWealthFactor,
                             df_curva: pd.DataFrame) -> Qcf.ZeroCouponCurve:
    """
    Retorna un objeto Qcf.ZeroCouponCurve. Esta función requiere que `df_curva` tenga una columna
    de nombre 'plazo' y una columna de nombre 'tasa'. Se usa interpolación lineal en la curva que
    se retorna.
    """
    plazos = Qcf.long_vec()
    tasas = Qcf.double_vec()
    for row in df_curva.itertuples():
        plazos.append(row.plazo)
        tasas.append(row.tasa)
    curva = Qcf.QCCurve(plazos, tasas)
    curva = Qcf.QCLinearInterpolator(curva)
    tipo_tasa = Qcf.QCInterestRate(0.0, yf, wf)
    curva = Qcf.ZeroCouponCurve(curva, tipo_tasa)
    return curva

In [26]:
df_curva.head().style.format(frmt)

,plazo,tasa,df
0,1,0.081111%,99.999778%
1,7,0.084051%,99.998388%
2,14,0.077967%,99.997010%
3,21,0.077358%,99.995549%
4,33,0.078067%,99.992942%


In [27]:
zcc = get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Algunos métodos del objeto`zcc`.

In [28]:
plazo = 900
print(f"Tasa a {plazo} días es igual a {zcc.get_rate_at(plazo):.4%}")
print(f"Factor de descuento a {plazo} días es igual a {zcc.get_discount_factor_at(plazo):.6%}")

Tasa a 900 días es igual a 0.0652%
Factor de descuento a 900 días es igual a 99.839384%


## Valorización

In [29]:
def get_ois_using_template(
    template,
    type_ois: TypeOis,
    rp: Qcf.RecPay,
    notional: float,
    start_date: Qcf.QCDate,
    tenor: Qcf.Tenor,
    fixed_rate_value: float,
    spread: float,
    gearing: float
):
    """
    """
    template_dict = template[type_ois]
    meses = tenor.get_years() * 12 + tenor.get_months()
    end_date = start_date.add_months(meses)
    template_dict['fixed_rate'].set_value(fixed_rate_value)
    es_bono = False

    # Construye la pata fija
    fixed_rate_leg = Qcf.LegFactory.build_bullet_fixed_rate_leg(
        rp,
        start_date,
        end_date,
        template_dict['bus_adj_rule'],
        template_dict['periodicity'],
        template_dict['stub_period'],
        get_cal(template_dict['calendar']),
        template_dict['settlement_lag'],
        notional,
        template_dict['amort_is_cashflow'],
        template_dict['fixed_rate'],
        template_dict['currency'],
        es_bono)

    # Construye la pata ois
    rp = Qcf.RecPay.PAY if rp == Qcf.RecPay.RECEIVE else Qcf.RecPay.RECEIVE
    icp_clp_leg = Qcf.LegFactory.build_bullet_icp_clp2_leg(
        rp,
        start_date,
        end_date,
        template_dict['bus_adj_rule'],
        template_dict['periodicity'],
        template_dict['stub_period'],
        get_cal(template_dict['calendar']),
        template_dict['settlement_lag'],
        notional,
        template_dict['amort_is_cashflow'],
        spread,
        gearing,
        True
    )

    for i in range(icp_clp_leg.size()):
        cshflw = icp_clp_leg.get_cashflow_at(i)
        cshflw.set_start_date_icp(1.0)
        cshflw.set_end_date_icp(1.0)

    return (fixed_rate_leg, icp_clp_leg)

### Operación Ejemplo

In [30]:
op = get_ois_using_template(
    type_ois_template,
    TypeOis.SOFR,
    Qcf.RecPay.RECEIVE,
    10000000,
    Qcf.QCDate(14, 10, 2020),
    Qcf.Tenor('2Y'),
    .01,
    0.0,
    1.0
)
op

(<finrisk.QC_Financial_3.Leg at 0x7fcfbbea3440>,
 <finrisk.QC_Financial_3.Leg at 0x7fcfbbea33d8>)

#### Digresión: `functools.partial`

Supongamos que estoy en una situación en la que sólo quiero construir OIS de *SOFR*. Me gustaría no tener que repetir los argumentos `type_ois_template` y `TypeOis.SOFR` cada vez que llamo la función `get_ois_using_template`. Puedo definir una nueva función de la siguiente forma:

In [31]:
get_ois_sofr = partial(get_ois_using_template, type_ois_template, TypeOis.SOFR)

Con esta nueva función, `get_ois_sofr`, ahora puedo construir la operación `op` de la siguiente forma:

In [32]:
op = get_ois_sofr(
    Qcf.RecPay.RECEIVE,
    10000000,
    Qcf.QCDate(14, 10, 2020),
    Qcf.Tenor('2Y'),
    .01,
    0.0,
    1.0
)

#### Continuamos (fin digresión ...)

In [33]:
my.leg_as_dataframe(op[0], my.TipoPata.FIJA).style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"10,000,000",0,"101,389",True,"101,388.8889",USD,1.0000%,LinAct360
1,2021-10-14,2022-10-14,2022-10-14,"10,000,000","10,000,000","101,389",True,"10,101,388.8889",USD,1.0000%,LinAct360


In [34]:
my.leg_as_dataframe(op[1], my.TipoPata.ICPCLP).style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"-10,000,000",0,True,0.0000,CLP,1.000000,1.000000,0.0000%,-0,0.0000%,1.000000,LinAct360
1,2021-10-14,2022-10-14,2022-10-14,"-10,000,000","-10,000,000",True,"-10,000,000.0000",CLP,1.000000,1.000000,0.0000%,-0,0.0000%,1.000000,LinAct360


#### Valor Presente Pata Fija

In [35]:
vp = Qcf.PresentValue()

In [36]:
fecha_val = Qcf.QCDate(14, 10, 2020)

In [21]:
vp_fija = vp.pv(fecha_val, op[0], zcc)
print(f'El valor presente de la pata fija es: USD {vp_fija:,.8f}')

El valor presente de la pata fija es: USD 10,191,249.70919298


**Ejercicio:** Replique el valor de la pata fija utilizando los flujos y los factores de descuento que obtiene de la curva `zcc`.

In [37]:
# Se dan de alta las fechas finales de ambos cupones
fecha1 = Qcf.QCDate(14, 10, 2021)
fecha2 = Qcf.QCDate(14, 10, 2022)

# O de forma más automática:
fecha1 = op[0].get_cashflow_at(0).get_settlement_date()
fecha2 = op[0].get_cashflow_at(1).get_settlement_date()

# Se calcula el número de días entre la fecha de valorización (fecha_val) y las fechas
# finales de ambos cupones.
plazo1 = fecha_val.day_diff(fecha1)
plazo2 = fecha_val.day_diff(fecha2)

# Utilizando la curva zcc se calculan los df a esos plazos 
df1 = zcc.get_discount_factor_at(plazo1)
df2 = zcc.get_discount_factor_at(plazo2)

# Se obtienen los flujos totales (interés y amortización) de ambos cupones
flujo1 = op[0].get_cashflow_at(0).amount()
flujo2 = op[0].get_cashflow_at(1).amount()

# Finalmente, se calcula el valor presente como el producto (escalar) entre los df y los flujos.
check_vp = df1 * flujo1 + df2 * flujo2

# Se muestra el resultado.
print(f'El valor presente a mano es: {check_vp:,.8f}')

El valor presente a mano es: 10,191,249.70919298


#### Valor Presente Pata Flotante

In [38]:
fwd = Qcf.ForwardRates()

In [39]:
print(f'VP: {vp.pv(fecha_val, op[1], zcc):,.2f}')

VP: -9,988,658.09


In [40]:
print(f'{df2 * 10000000:,.2f}')

9,988,658.09


In [41]:
fwd.set_rates_icp_clp_leg(fecha_val, 1.0, op[1], zcc)

In [42]:
my.leg_as_dataframe(op[1], my.TipoPata.ICPCLP).style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2020-10-14,2021-10-14,2021-10-14,"-10,000,000",0,True,"-7,023.7778",CLP,1.000000,1.000702,0.0700%,"-7,097",0.0000%,1.000000,LinAct360
1,2021-10-14,2022-10-14,2022-10-14,"-10,000,000","-10,000,000",True,"-10,004,327.9717",CLP,1.000702,1.001135,0.0400%,"-4,056",0.0000%,1.000000,LinAct360


In [43]:
vp_flot = vp.pv(fecha_val, op[1], zcc)
print(f'El valor presente de la pata flotante es: USD {vp_flot:,.2f}')

El valor presente de la pata flotante es: USD -10,000,000.00


¿Porqué sabíamos que tenía que dar 10,000,000.00? Hint: la respuesta está en la construcción de la curva cero OIS.

## Sensibilidad

En `QC_Financial_3` al calcular el valor presente, también se calculan las derivadas del valor presente respecto a cada uno de los vértices de la curva.

### Pata Fija

In [44]:
vp.pv(fecha_val, op[0], zcc)
der = vp.get_derivatives()

Con esas derivadas, se puede calcular la sensibilidad a cada vértice de la curva cupón cero para un movimiento de 1 punto básico.

In [30]:
print(type(der))

<class 'finrisk.QC_Financial_3.double_vec'>


In [46]:
delta = .0001
total = 0
for i, d in enumerate(der):
    total += d * delta
    print(f"Sensibilidad en {i}: {d * delta: 0,.2f}")
print(f"Sensibilidad total: {total:,.2f}")

Sensibilidad en 0:  0.00
Sensibilidad en 1:  0.00
Sensibilidad en 2:  0.00
Sensibilidad en 3:  0.00
Sensibilidad en 4:  0.00
Sensibilidad en 5:  0.00
Sensibilidad en 6:  0.00
Sensibilidad en 7:  0.00
Sensibilidad en 8:  0.00
Sensibilidad en 9:  0.00
Sensibilidad en 10:  0.00
Sensibilidad en 11:  0.00
Sensibilidad en 12:  0.00
Sensibilidad en 13:  0.00
Sensibilidad en 14:  0.00
Sensibilidad en 15: -10.12
Sensibilidad en 16:  0.00
Sensibilidad en 17: -2,016.84
Sensibilidad en 18:  0.00
Sensibilidad en 19:  0.00
Sensibilidad en 20:  0.00
Sensibilidad en 21:  0.00
Sensibilidad en 22:  0.00
Sensibilidad en 23:  0.00
Sensibilidad en 24:  0.00
Sensibilidad en 25:  0.00
Sensibilidad en 26:  0.00
Sensibilidad en 27:  0.00
Sensibilidad en 28:  0.00
Sensibilidad en 29:  0.00
Sensibilidad en 30:  0.00
Sensibilidad en 31:  0.00
Sensibilidad en 32:  0.00
Sensibilidad total: -2,026.97


**Ejercicio:** Verifique por diferencias finitas centrales la sensibilidad en el vértice 17. La aproximación de la derivada por diferencias finitas centrales es:

$$
\begin{equation}
f^{\prime}\left(x\right)\approx\frac{f\left(x+h\right)-f\left(x-h\right)}{2h}
\end{equation}
$$

### Pata Flotante

In [47]:
vp.pv(fecha_val, op[1], zcc)
der = vp.get_derivatives()

In [48]:
total = 0
for i, d in enumerate(der):
    total += d * delta
    print(f"Sensibilidad en {i}: {d * delta: 0,.2f}")
print(f"Sensibilidad total: {total:,.2f}")

Sensibilidad en 0:  0.00
Sensibilidad en 1:  0.00
Sensibilidad en 2:  0.00
Sensibilidad en 3:  0.00
Sensibilidad en 4:  0.00
Sensibilidad en 5:  0.00
Sensibilidad en 6:  0.00
Sensibilidad en 7:  0.00
Sensibilidad en 8:  0.00
Sensibilidad en 9:  0.00
Sensibilidad en 10:  0.00
Sensibilidad en 11:  0.00
Sensibilidad en 12:  0.00
Sensibilidad en 13:  0.00
Sensibilidad en 14:  0.00
Sensibilidad en 15:  0.70
Sensibilidad en 16:  0.00
Sensibilidad en 17:  1,997.46
Sensibilidad en 18:  0.00
Sensibilidad en 19:  0.00
Sensibilidad en 20:  0.00
Sensibilidad en 21:  0.00
Sensibilidad en 22:  0.00
Sensibilidad en 23:  0.00
Sensibilidad en 24:  0.00
Sensibilidad en 25:  0.00
Sensibilidad en 26:  0.00
Sensibilidad en 27:  0.00
Sensibilidad en 28:  0.00
Sensibilidad en 29:  0.00
Sensibilidad en 30:  0.00
Sensibilidad en 31:  0.00
Sensibilidad en 32:  0.00
Sensibilidad total: 1,998.16


La estructura es la misma que para una pata fija, lo que indica que se debe también incluir la sensibilidad a la curva de proyección.

In [52]:
import numpy as np
result = []
for i in range(op[1].size()):
    # Se obtiene cada uno de los flujos
    cshflw = op[1].get_cashflow_at(i)
    
    # Se obtiene el gradiente de los flujos respecto a la curva de proyección
    amt_der = cshflw.get_amount_derivatives()
    
    # Se obtiene el factor de descuento que corresponde al flujo
    df = zcc.get_discount_factor_at(fecha_val.day_diff(cshflw.get_settlement_date()))
    
    # Con el gradiente y el factor de descuento se obtiene la sensibilidad
    amt_sens = [da_dz * delta * df for da_dz in amt_der]
    
    # Se almacena la sensibilidad
    if len(amt_sens) > 0:
        result.append(np.array(amt_sens))
        
# Vector donde se almacena la sensibilidad total por vértice
total = result[0] * 0

# Esta es una suma vectorial componente a componente
for r in result:
    total += r

# Se muestra el resultado
for i in range(len(total)):
    print(f"Sensibilidad en {i}: {total[i]:0,.2f}")

print(f"Sensibilidad de proyección: {sum(total):,.2f} USD")

Sensibilidad en 0: -0.00
Sensibilidad en 1: -0.00
Sensibilidad en 2: -0.00
Sensibilidad en 3: -0.00
Sensibilidad en 4: -0.00
Sensibilidad en 5: -0.00
Sensibilidad en 6: -0.00
Sensibilidad en 7: -0.00
Sensibilidad en 8: -0.00
Sensibilidad en 9: -0.00
Sensibilidad en 10: -0.00
Sensibilidad en 11: -0.00
Sensibilidad en 12: -0.00
Sensibilidad en 13: -0.00
Sensibilidad en 14: -0.00
Sensibilidad en 15: -0.70
Sensibilidad en 16: -0.00
Sensibilidad en 17: -1,997.46
Sensibilidad en 18: -0.00
Sensibilidad en 19: -0.00
Sensibilidad en 20: -0.00
Sensibilidad en 21: -0.00
Sensibilidad en 22: -0.00
Sensibilidad en 23: -0.00
Sensibilidad en 24: -0.00
Sensibilidad en 25: -0.00
Sensibilidad en 26: -0.00
Sensibilidad en 27: -0.00
Sensibilidad en 28: -0.00
Sensibilidad en 29: -0.00
Sensibilidad en 30: -0.00
Sensibilidad en 31: -0.00
Sensibilidad en 32: -0.00
Sensibilidad de proyección: -1,998.16 USD


**Ejercicio:** Ambas sensibilidades se cancelan. ¿Porqué?